# Лабораторная работа №3. Методы поиска подстроки в строке.
## Выполнил студент группы БСТ1904 Колотилов Д.А.

## Задание 1

Реализовать методы поиска подстроки в строке. Добавить возможность
ввода строки и подстроки с клавиатуры. Предусмотреть возможность
существования пробела. Реализовать возможность выбора опции
чувствительности или нечувствительности к регистру. Оценить время работы
каждого алгоритма поиска и сравнить его со временем работы стандартной
функции поиска, используемой в выбранном языке программирования.

**Алгоритмы:**

1. Кнута-Морриса-Пратта
2. Упрощенный Бойера-Мура

## Алгоритм Кнута-Морриса-Пратта

In [6]:
# Префикс функция для исходной строки, где k - длина наибольшего собственного префикса подстроки s[0:i]
def prefix(s):
    # Массив из нулей, в котором будут находиться длины
    p = [0] * len(s)
    # Проходим по всей строке и смотрим длины префиксов и суффиксов
    for i in range(1, len(s)):
        # Берем наибольшую длину
        k = p[i - 1]
        # Пока длина больше 0 и элементы не равны, будем менять значение длины в меньшую сторону
        while k > 0 and s[k] != s[i]:
            k = p[k - 1]
        # Если элементы равны, то увеличиваем длину
        if s[k] == s[i]:
            k += 1
        # Обновляем значение длины по индексу
        p[i] = k
    print(p)
    return p

In [7]:
def KMP(s, w):
    # Массив A - массив, в котором будут храниться все места, где есть данный образ
    A = []
    # Длина префикса
    k = 0
    # Пропускаем строку через префикс-функцию
    p = prefix(s)
    # Проходим по длине всей строки и используем алгоритм префикс-функции
    for i in range(len(s)):
        while k > 0 and s[i] != w[k]:
            k = p[k - 1]
        if s[i] == w[k]:
            k += 1
        # Если длина образа совпадает с длиной префикса, то добавляем в массив место, где нашли образ в строке
        if k == len(w):
            A.append([i - len(w) + 1, i + 1])
            k = p[k - 1]

    return A

In [8]:
print(KMP("aabaabaaaabaabaaab", "aabaa"))

[0, 1, 0, 1, 2, 3, 4, 5, 2, 2, 3, 4, 5, 6, 7, 8, 9, 3]
[[0, 5], [3, 8], [8, 13], [11, 16]]


## Упрощенный алгоритм Бойера-Мура

In [90]:
# Строит таблицу смещений для образа
def preprocess(w):
    T = [len(w)] * 256
    for i in range(len(w) - 1):
        T[ord(w[i])] = len(w) - 1 - i
    return T

In [91]:
# Алгоритм Бойера-Мура
def BM(s, w):
    # Массив с местами, где был найден образ
    A = []
    # Строим таблицу смещений для образа
    T = preprocess(w)
    # Переменная для пропуска
    skip = 0
    # Пока длина строки без пропуска больше или равна образцу
    while len(s) - skip >= len(w):
        # Если данный отрезок равен образцу, то добавляем его в массив
        if s[skip:skip + len(w)] == w:
            A.append([skip, skip + len(w)])
        # Увеличиваем пропуск
        skip += T[ord(s[skip + len(w) - 1])]
    return A

In [92]:
print(BM("Hoola-Hoola girls like Hooligans", "Hooligan"))

[[23, 31]]


## Поиск подстроки с возможностью игнорирования пробелов и регистра

In [40]:
def search(s, w, fn=KMP, ignore_case=True, ignore_space=True):
    _s = s
    _w = w
    if ignore_case:
        _s = _s.lower()
        _w = _w.lower()
    
    if ignore_space:
        _s = _s.replace(' ', '')
        _w = _w.replace(' ', '')

    A = fn(_s, _w)

    if ignore_space:
        nonspace = 0
        kmp_without_space = 0
        index = 0
        while kmp_without_space < len(A) and index < len(s):
            if A[kmp_without_space][0] == nonspace:
                index_with_space_offsets = index
                chars_count = 0
                while chars_count < len(_w) and index_with_space_offsets < len(s):
                    if s[index_with_space_offsets] != ' ':
                        chars_count += 1
                    index_with_space_offsets += 1
                A[kmp_without_space] = (index, index_with_space_offsets)
                kmp_without_space += 1
            if s[index] != ' ': nonspace += 1
            index += 1
    
    return A

In [34]:
search("aabaabaaaabaabaaab", "aabaa")

[0, 1, 0, 1, 2, 3, 4, 5, 2, 2, 3, 4, 5, 6, 7, 8, 9, 3]


[(0, 5), (3, 8), (8, 13), (11, 16)]

## Задание 2 «Пятнашки»

In [100]:
from queue import PriorityQueue

N = 4

# Движение пятнашек
def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

# Функция для определения есть решение или нет
def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2

# Класс позиции
class Position:
    # Конструктор класса, который принимает позицию и начальную дистанцию
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    # Метод, который срабатывает при сравнении (<) объектта с другим объектом
    def __lt__(self, other):
        return self.start_distance < other.start_distance

    # Метод, который срабатывает при использовании объекта как строки
    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

# Разгадка
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
# Загадка
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

# Смотрим, можно ли в данной расстановке найти решение
# Если нет, то сообщаем об этом
if parity(start) == 0:
    print('Нерешаемо')
# Иначе ищем этот путь
else:
    # Кортеж загадки
    start = tuple(start)
    
    # Первоначальная позиция
    p = Position(start, 0)
    print("Первоначальная позиция: " + "\n", p)
    print()
    
    # 1) Кладем в очередь с приоритетом первоначальную позицию
    candidates = PriorityQueue()
    candidates.put(p)

    # Кортеж посещенных позиций
    visited = set([p])

    # Откуда пришли
    came_from = {p.position: None}
    
    # Пока решение не найдено
    while p.position != SOLVED:
        # 2) Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        # 3) Кладем в очередь все соседние позиции
        # 4) Повторяем пункты 2-4 пока в пункте 2 не вытащим конечную позицию
        for k in moves(p.position):
            if k not in visited:
                # В candifates хранятся всевозможные позиции
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    # path - последовательное решение головоловки (путь)
    path = []
    # Сохраняем конечную позицию
    prev = p
    # Идем в обратном порядке и запоминаем очередность хода в path
    while p.position != start:
        # Запоминаем откуда ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print("Оптимальный путь к решению:" + "\n", path)

Первоначальная позиция: 
   1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

Оптимальный путь к решению:
 [15, 14, 10, 13, 9, 10, 14, 15]


## Вывод
В данной лабораторной работе я изучил методы поиска образа в строке